In [1]:
import sys
import gzip

import numpy as np
import tensorflow as tf

from collections import namedtuple

from binary_sayc import load_deals, DealData
from simulator import Simulator, get_potential_contracts
from eval_bidding import eval_bidding

In [2]:
sess = tf.InteractiveSession()
# saver = tf.train.import_meta_graph('../model/sayc-500000.meta')
# saver.restore(sess, '../model/sayc-500000')
saver = tf.train.import_meta_graph('./jos_model/jos-500000.meta')
saver.restore(sess, './jos_model/jos-500000')

lstm_size = 128

graph = tf.get_default_graph()

seq_in = graph.get_tensor_by_name('seq_in:0')
seq_out = graph.get_tensor_by_name('seq_out:0')
keep_prob = graph.get_tensor_by_name('keep_prob:0')

out_bid_logit = graph.get_tensor_by_name('out_bid_logit:0')
out_bid_target = graph.get_tensor_by_name('out_bid_target:0')

state_c_0 = graph.get_tensor_by_name('state_c_0:0')
state_h_0 = graph.get_tensor_by_name('state_h_0:0')

state_c_1 = graph.get_tensor_by_name('state_c_1:0')
state_h_1 = graph.get_tensor_by_name('state_h_1:0')

state_c_2 = graph.get_tensor_by_name('state_c_2:0')
state_h_2 = graph.get_tensor_by_name('state_h_2:0')

next_c_0 = graph.get_tensor_by_name('next_c_0:0')
next_h_0 = graph.get_tensor_by_name('next_h_0:0')

next_c_1 = graph.get_tensor_by_name('next_c_1:0')
next_h_1 = graph.get_tensor_by_name('next_h_1:0')

next_c_2 = graph.get_tensor_by_name('next_c_2:0')
next_h_2 = graph.get_tensor_by_name('next_h_2:0')

x_in = graph.get_tensor_by_name('x_in:0')
out_bid = graph.get_tensor_by_name('out_bid:0')

INFO:tensorflow:Restoring parameters from ./jos_model/jos-500000


In [3]:
State = namedtuple('State', ['c', 'h'])

zero_state = (
    State(c=np.zeros((1, lstm_size)), h=np.zeros((1, lstm_size))),
    State(c=np.zeros((1, lstm_size)), h=np.zeros((1, lstm_size))),
    State(c=np.zeros((1, lstm_size)), h=np.zeros((1, lstm_size))),
)

nesw_initial = [zero_state, zero_state, zero_state, zero_state]

def model(sess, p_keep=1.0):
    def pred_fun(x, state_in):
        feed_dict = {
            keep_prob: p_keep,
            x_in: x,
            state_c_0: state_in[0].c,
            state_h_0: state_in[0].h,
            state_c_1: state_in[1].c,
            state_h_1: state_in[1].h,
            state_c_2: state_in[2].c,
            state_h_2: state_in[2].h,
        }
        bids = sess.run(out_bid, feed_dict=feed_dict)
        next_state = (
            State(c=sess.run(next_c_0, feed_dict=feed_dict), h=sess.run(next_h_0, feed_dict=feed_dict)),
            State(c=sess.run(next_c_1, feed_dict=feed_dict), h=sess.run(next_h_1, feed_dict=feed_dict)),
            State(c=sess.run(next_c_2, feed_dict=feed_dict), h=sess.run(next_h_2, feed_dict=feed_dict)),
        )
        return bids, next_state
    return pred_fun

In [12]:
graph.get_tensor_by_name('out_rnn:0').shape

KeyError: "The name 'out_rnn:0' refers to a Tensor which does not exist. The operation, 'out_rnn', does not exist in the graph."

In [6]:
cost = tf.losses.softmax_cross_entropy(out_bid_target, out_bid_logit)

#train_step = tf.train.AdamOptimizer(0.0001, name='online').minimize(cost)
train_step = graph.get_operation_by_name('Adam')

In [4]:
reader = load_deals(gzip.open('../deals_bidding_valid_0001.gz'))

In [6]:
for i, (deal_str, auction_str, contracts) in enumerate(reader):
    if i % 10 == 0:
        sys.stdout.write('.')
#     if i > 1000:
#         break
    deal_data = DealData.from_deal_auction_string(deal_str, auction_str)
    deal_data.reset_auction()
    init_auction = deal_data.auction
    
    sim = Simulator(deal_data, contracts, model(sess, 1.0))
    
    best_auction, best_score = sim.best(
        init_auction, 
        nesw_initial,
        -10000,
        10000,
        get_potential_contracts(contracts, 0, int(deal_data.vuln_ns)),
        get_potential_contracts(contracts, 1, int(deal_data.vuln_ew))
    )
    
    deal_data.auction = best_auction
    
    n_time_steps = (len(best_auction) // 4) + 1
    
    X, y = deal_data.get_binary(n_time_steps)
    
    sess.run(train_step, feed_dict={seq_in: X, seq_out: y, keep_prob: 0.8})

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [5]:
def apply_bidding(bid_model, fin, fout):
    reader = load_deals(fin)
    for i, (deal_str, auction_str, contracts) in enumerate(reader):
        if i % 10 == 0:
            sys.stdout.write('.')
        if i > 1000:
            break
        deal_data = DealData.from_deal_auction_string(deal_str, auction_str)
        deal_data.reset_auction()
        init_auction = deal_data.auction
        sim = Simulator(deal_data, contracts, bid_model)
        auction = [('P' if bid == 'PASS' else bid) for bid in sim.simulate_auction(init_auction, nesw_initial, True) if bid != 'PAD_START']
        fout.write((' ' + deal_str + '\n').encode('ascii'))
        fout.write((' ' + ' '.join(auction_str.split()[:2] + auction)).encode('ascii'))
        fout.write('\n'.encode('ascii'))
        for contract, (score1, score2) in contracts.items():
            fout.write(('{}\t{}\t{}\n'. format(contract, score1, score2)).encode('ascii'))

In [6]:
fout = gzip.open('../test.gz', 'w')
apply_bidding(model(sess), gzip.open('../deals_bidding_valid_0001.gz'), fout)
fout.close()

.....................................................................................................

In [7]:
eval_bidding(gzip.open('../test.gz'))

..

231.55844155844156

In [6]:
m0 = model(sess, 1.0)

In [19]:
deal_str, auction_str, contracts = next(reader)

In [20]:
deal_data = DealData.from_deal_auction_string(deal_str, auction_str)
deal_data.reset_auction()
init_auction = deal_data.auction
sim = Simulator(deal_data, contracts, m0)

In [21]:
print(deal_str)
print(auction_str)

SJ8754 HAJ65 DK73 CT  SAQ6 HQ8 DT2 CAQ7632  SKT9 HKT92 DQ5 CK985  S32 H743 DAJ9864 CJ4
S E-W P P P 1C P 1D P 2C P 2D P P P


In [25]:
dlr_vuln = auction_str.split(' ')
disp_deal(deal_str, dlr_vuln[0], dlr_vuln[1])
print('\nJOS NN bidding:')
disp_auction(sim.simulate_auction(init_auction, nesw_initial, max_bid=True))

S E-W   J8754
        AJ65
        K73
        T
32              AQ6
743             Q8
AJ9864          T2
J4              AQ7632
        KT9
        KT92
        Q5
        K985

JOS NN bidding:
West North  East South
                  p 
p     p     1C    p 
1D    1S    2C    2S
p     p     3C    p 
p     p 


In [18]:
sim.par

'5SXE'

In [107]:
contracts

{'1CE': (-150, -300),
 '1CN': (110, 110),
 '1CS': (110, 110),
 '1CW': (-150, -300),
 '1CXE': (-500, -800),
 '1CXN': (340, 540),
 '1CXS': (340, 540),
 '1CXW': (-500, -800),
 '1CXXE': (-1000, -1600),
 '1CXXN': (630, 1030),
 '1CXXS': (630, 1030),
 '1CXXW': (-1000, -1600),
 '1DE': (-150, -300),
 '1DN': (90, 90),
 '1DS': (90, 90),
 '1DW': (-150, -300),
 '1DXE': (-500, -800),
 '1DXN': (240, 340),
 '1DXS': (240, 340),
 '1DXW': (-500, -800),
 '1DXXE': (-1000, -1600),
 '1DXXN': (430, 630),
 '1DXXS': (430, 630),
 '1DXXW': (-1000, -1600),
 '1HE': (-50, -100),
 '1HN': (-50, -100),
 '1HS': (-50, -100),
 '1HW': (-50, -100),
 '1HXE': (-100, -200),
 '1HXN': (-100, -200),
 '1HXS': (-100, -200),
 '1HXW': (-100, -200),
 '1HXXE': (-200, -400),
 '1HXXN': (-200, -400),
 '1HXXS': (-200, -400),
 '1HXXW': (-200, -400),
 '1NE': (-150, -300),
 '1NN': (120, 120),
 '1NS': (120, 120),
 '1NW': (-150, -300),
 '1NXE': (-500, -800),
 '1NXN': (280, 380),
 '1NXS': (280, 380),
 '1NXW': (-500, -800),
 '1NXXE': (-1000, -160

In [23]:
call_trans = {
    'PASS': 'p ',
    'X': 'X ',
    'PAD_START': '  '
}
def disp_auction(auction):
    auction = ['  '] + [call_trans[call] if call in call_trans else call for call in auction]
    print(' '.join(['West', 'North ', 'East', 'South']))
    acc = []
    for i, call in enumerate(auction):
        
        if i % 4 == 0 and i > 0:
            print('    '.join(acc))
            acc = []
        acc.append(call)
    print('    '.join(acc))

In [24]:
disp_auction(['PASS', '3D', '3S', 'PASS', '4S', 'PASS', 'PASS', 'PASS'])

West North  East South
      p     3D    3S
p     4S    p     p 
p 


In [11]:
def disp_deal(deal_str, dealer, vuln):
    hands = deal_str.split('  ')
    north = [s[1:] for s in hands[0].split(' ')]
    east = [s[1:] for s in hands[1].split(' ')]
    south = [s[1:] for s in hands[2].split(' ')]
    west = [s[1:] for s in hands[3].split(' ')]
    annot = dealer + ' ' + vuln
    print(annot + (8 - len(annot)) * ' ' + north[0])
    print(8 * ' ' + north[1])
    print(8 * ' ' + north[2])
    print(8 * ' ' + north[3])
    print(west[0] + (16 - len(west[0])) * ' ' + east[0])
    print(west[1] + (16 - len(west[1])) * ' ' + east[1])
    print(west[2] + (16 - len(west[2])) * ' ' + east[2])
    print(west[3] + (16 - len(west[3])) * ' ' + east[3])
    print(8 * ' ' + south[0])
    print(8 * ' ' + south[1])
    print(8 * ' ' + south[2])
    print(8 * ' ' + south[3])

In [55]:
disp_deal('SAK4 H92 D9875 CQ952  S62 HK87 DAQJT43 C87  SQJT75 HAQ4 D6 CAKT3  S983 HJT653 DK2 CJ64', 'N', 'N-S')

N N-S   AK4
        92
        9875
        Q952
983             62
JT653           K87
K2              AQJT43
J64             87
        QJT75
        AQ4
        6
        AKT3


In [76]:
sim.best(
    init_auction, 
    nesw_initial,
    -10000,
    10000,
    get_potential_contracts(contracts, 0, int(deal_data.vuln_ns)),
    get_potential_contracts(contracts, 1, int(deal_data.vuln_ew))
)

(['PAD_START',
  'PAD_START',
  'PASS',
  'PASS',
  '2N',
  'PASS',
  '3C',
  'PASS',
  '3S',
  'PASS',
  '4N',
  'PASS',
  '5D',
  'PASS',
  '5H',
  'PASS',
  '5N',
  'PASS',
  '6S',
  'PASS',
  'PASS',
  'PASS'],
 1430)

In [11]:
sim.cache={}; sim.simulate_auction(init_auction, nesw_initial, True)

['1H', 'PASS', '2N', 'PASS', '3S', 'PASS', '4H', 'PASS', 'PASS', 'PASS']

In [16]:
out_bid

array([[  1.64527891e-16,   5.99772940e-11,   6.34882497e-07,
          7.25274618e-10,   1.73577988e-16,   1.81270252e-06,
          2.19860885e-09,   9.99997139e-01,   1.10037277e-07,
          1.70532473e-07,   2.97361300e-08,   1.27333474e-11,
          3.65180703e-08,   1.21770118e-12,   2.90017859e-08,
          5.84454263e-10,   3.73907245e-14,   2.24436317e-10,
          8.33857244e-16,   8.17655351e-13,   2.10195437e-11,
          2.84233340e-15,   1.63448283e-11,   4.94362945e-15,
          9.82323055e-13,   1.36874720e-10,   9.88046775e-13,
          2.03621473e-12,   6.45679392e-14,   4.27406287e-12,
          3.00925153e-11,   7.49699348e-12,   1.09034913e-10,
          9.85024237e-14,   1.47835771e-10,   4.19949686e-14,
          4.57545705e-16,   8.55373105e-14,   1.63578632e-15,
          1.21645001e-12]], dtype=float32)

In [33]:
init_auction

['PAD_START']

In [34]:
out_bid_np, next_state_np = sim.next_bid_np(init_auction, nesw_initial[0])

In [35]:
out_bid_np

array([[  1.53094361e-14,   5.54510279e-07,   9.99850750e-01,
          8.58317071e-06,   1.47582819e-14,   1.21368475e-06,
          1.14991803e-06,   1.61636399e-06,   1.11221188e-05,
          8.50125070e-06,   1.15213488e-05,   2.88907886e-05,
          7.02543821e-06,   6.68111988e-06,   2.92251570e-05,
          2.95421705e-05,   4.72016325e-07,   1.91950789e-07,
          6.43002451e-08,   9.89321279e-07,   3.13422333e-07,
          1.59872815e-09,   6.18585219e-08,   2.97956007e-07,
          8.20846591e-09,   7.83959138e-07,   3.84520638e-09,
          3.58623048e-10,   3.38638007e-09,   5.93815237e-08,
          5.20209431e-10,   5.08801577e-11,   4.47270609e-10,
          7.66769848e-09,   7.34651309e-08,   1.03195201e-14,
          4.61926368e-14,   1.22355824e-14,   5.66635367e-12,
          2.52916937e-10]], dtype=float32)

In [36]:
sim_auction, sim_out_bid = sim.simulate_auction_np(init_auction, nesw_initial)

In [37]:
sim_auction

['PAD_START', 'PASS', '1H', '2S', 'PASS', '4S', 'PASS', 'PASS', 'PASS']

In [13]:
len(sim_auction), len(sim_out_bid)

(10, 10)

In [14]:
for sim_bid, out_bid_np in zip(sim_auction, sim_out_bid):
    print(sim_bid, np.max(out_bid_np))

1H 0.999997
PASS 0.999993
2N 0.99096
PASS 0.999818
3S 0.764141
PASS 1.0
4H 0.940278
PASS 0.999998
PASS 0.999408
PASS 1.0


In [38]:
uncertain = 1 - np.array([1] * (len(sim_auction) - len(sim_out_bid)) + [np.max(out_bid_np) for out_bid_np in sim_out_bid])

In [39]:
uncertain = uncertain / (np.sum(uncertain) + 1e-6)
uncertain

array([  0.00000000e+00,   3.38339754e-04,   1.34660303e-03,
         2.56711370e-01,   4.61749179e-01,   1.68438069e-01,
         2.43215478e-06,   1.01535571e-01,   9.87616984e-03])

In [40]:
np.sum(uncertain)

0.99999773306743955

In [41]:
np.random.multinomial(1, uncertain)

array([0, 0, 0, 0, 1, 0, 0, 0, 0])

In [24]:
np.sum(uncertain)

1.0

In [45]:
np.random.multinomial(1, [0.1, 0.2, 0.7])

array([0, 0, 1])

In [62]:
sim_auction, sim_out_bid = sim.simulate_auction_np(init_auction, nesw_initial, 6)
sim_auction

['PAD_START',
 'PASS',
 '1H',
 '2S',
 'PASS',
 '4S',
 'PASS',
 '5S',
 'PASS',
 'PASS',
 'PASS']